In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
google_api_key=os.getenv("GEMINI_API_KEY") #to get key

In [6]:
if google_api_key=="":
    print("API key not found")
else:
    print("API key found")

API key found


In [ ]:

from llama_index.llms.gemini import Gemini

import google.generativeai as genai

from llama_index.core import SimpleDirectoryReader #used to load data

from llama_index.core import VectorStoreIndex

from IPython.display import Markdown, display

from llama_index.core import ServiceContext

from llama_index.core import StorageContext, load_index_from_storage

from llama_index.embeddings.gemini import GeminiEmbedding

/Users/mahikapatney/Desktop/QAsystem/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
genai.configure(api_key=google_api_key) #to check if api is working or not

In [12]:
#checing if i can call my models or not using the key
for models in genai.list_models():
    print(models)


Model(name='models/chat-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 Chat (Legacy)',
      description='A legacy text-only model optimized for chat conversations',
      input_token_limit=4096,
      output_token_limit=1024,
      supported_generation_methods=['generateMessage', 'countMessageTokens'],
      temperature=0.25,
      max_temperature=None,
      top_p=0.95,
      top_k=40)
Model(name='models/text-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 (Legacy)',
      description='A legacy model that understands text and generates text as an output',
      input_token_limit=8196,
      output_token_limit=1024,
      supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
      temperature=0.7,
      max_temperature=None,
      top_p=0.95,
      top_k=40)
Model(name='models/embedding-gecko-001',
      base_model_id='',
      version='001',
      display_name='Embedding Gecko

In [13]:
for models in genai.list_models():
  if 'generateContent' in models.supported_generation_methods:
    print(models.name)
#some of these models are text- to- text generation and image-to-text generation

models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinki

In [14]:
documents=SimpleDirectoryReader("../Data")

In [15]:
doc=documents.load_data()

In [16]:
print(doc[0].text)

Proceedings of NAACL-HLT 2019: Demonstrations, pages 54–59
Minneapolis, Minnesota, June 2 - June 7, 2019.c⃝2019 Association for Computational Linguistics
54
FLAIR : An Easy-to-Use Framework for State-of-the-Art NLP
Alan Akbik† Tanja Bergmann†∗ Duncan Blythe†⋆
Kashif Rasul† Stefan Schweter‡
†Zalando Research, Mühlenstraße 25, 10243 Berlin
‡Bayerische Staatsbibliothek München, Digital Library/Munich Digitization Center, 80539 Munich
⋆Aleph-One GmbH, Rigaer Straße 8, 12047 Berlin ∗RASA AI, Berlin
Roland Vollgraf†
Abstract
We present F LAIR , an NLP framework de-
signed to facilitate training and distribution of
state-of-the-art sequence labeling, text classi-
ﬁcation and language models. The core idea
of the framework is to present a simple, uni-
ﬁed interface for conceptually very different
types of word and document embeddings. This
effectively hides all embedding-speciﬁc engi-
neering complexity and allows researchers to
“mix and match” various embeddings with lit-
tle effort. The fram

In [ ]:
model=Gemini(models='gemini-pro',api_key=google_api_key) #main model is gemini-pro and under that we are using embedding-001

/var/folders/sh/zthspjyd3pqcz2d6zydgmcm00000gp/T/ipykernel_1513/2941566521.py:1: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  model=Gemini(models='gemini-pro',api_key=google_api_key)


In [18]:
gemini_embed_model=GeminiEmbedding(model_name="models/embedding-001")

/var/folders/sh/zthspjyd3pqcz2d6zydgmcm00000gp/T/ipykernel_1513/4148756438.py:1: DeprecationWarning: Call to deprecated class GeminiEmbedding. (Should use `llama-index-embeddings-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/embeddings/google_genai/)
  gemini_embed_model=GeminiEmbedding(model_name="models/embedding-001")


In [38]:
# # Configure Service Context- its a container that contains the following objects-index, query,helps in retrieving based on similar context(helps in indexing the embedding)
# import llama_index
# from llama_index.core import VectorStoreIndex
# from llama_index.llms.ollama import Ollama
# from llama_index.embeddings.gemini import GeminiEmbedding
# from llama_index.core import Settings
# from llama_index.core.node_parser import SentenceSplitter

# # Assuming you've already defined this:
# gemini_embed_model = GeminiEmbedding(model_name="models/embedding-001")

# # Configure LLM, embedding model, and node parser globally
# Settings.llm = Ollama(model="llama2", request_timeout=120.0)
# Settings.embed_model = gemini_embed_model
# Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
# index = VectorStoreIndex.from_documents(doc)

# Configure Service Context using Settings
Settings.llm = model
Settings.embed_model = gemini_embed_model
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)

# Build index and query
index = VectorStoreIndex.from_documents(doc)
index.storage_context.persist()
query_engine = index.as_query_engine()

In [39]:
response = query_engine.query("what is a FLAIR?")
print(response)

Embeddings are the core concept of FLAIR. Each embedding class implements either the TokenEmbedding or the DocumentEmbedding interface for word and document embeddings respectively. Both interfaces define the .embed() method to embed a Sentence or a list of Sentence objects into a specific embedding space.



In [40]:

response = query_engine.query("what is the command for install the FLAIR library?")
print(response)

The command to install the FLAIR library is: pip install flair.



In [42]:


response = query_engine.query("what is the proposed solution from this paper?")
print(response)

The framework is being extended in multiple directions with the help of the open source community. This includes plans to support more embedding approaches like transformer embeddings, InferSent representations and LASER embeddings, and to expand coverage of NLP datasets and formats for automatic data fetching. Current research is also focused on developing new embedding types.



In [44]:

response = query_engine.query("what github links of this framework?")
print(response)

The provided text does not contain any GitHub links related to the framework.



In [45]:

response = query_engine.query("what is the minimum python version to setup FLAIR in the environment?")
print(response)

A current version of Python (at least version 3.6) is required to set up FLAIR.



In [46]:

response = query_engine.query("What are the presented framework used for?")
print(response)

The framework is being extended to support more embedding approaches and expand coverage of NLP datasets and formats for automatic data fetching. Current research also focuses on developing new embedding types and investigating further.

